#### Importing Libraries

In [1]:
# Import necessary libraries 
import os, json
import pandas as pd
import numpy as np

#### Setting up Paths and API Keys

In [2]:
# Establish base and report directories
base_path = os.path.abspath('.')
reports_path = os.path.join(base_path, 'cuckoo')

<br/><br/>

-----------

<br/><br/>

#### Parsing Cuckoo JSON Reports

In [3]:
# Initialize a list to store parsed cuckoo report data
cuckoo_reports = []

# Iterate over each item in the reports directory
for task_id_name in os.listdir(reports_path):
  # Skip non-directory items.
  if not os.path.isdir(os.path.join(reports_path, task_id_name)):
    print(f"Skipping {task_id_name} as it is not a folder")
    continue

  # Skip directories missing a JSON report
  if not os.path.exists(os.path.join(reports_path, task_id_name, 'reports', 'report.json')):
    print(f"Skipping {task_id_name} as it does not have a report.json file")
    continue
  
  # Load and parse the JSON report
  with open(os.path.join(reports_path, task_id_name, 'reports', 'report.json')) as f:
    report = json.loads(f.read())
  
  # Extract relevant information from the report
  cuckoo_report = {
      'task_id': task_id_name,
      'score': report['info']['score'],
      'signatures_count' : len(report['signatures']),
      'signature_mark_call_count': sum(sum( len([y for y in x if 'call' in x.keys() ])  for x in sig['marks'] if 'marks' in sig.keys() ) for sig in report['signatures']),
      'category': report['info']['category'],
      'package': report['info']['package'],
      'strings_count' : len(set(report['strings'])),
  }
  
  # Include optional details if available
  for key in ['static', 'fileops']:
    if key in report:
      cuckoo_report[key] = report[key]
  
  # Include behavior details if available
  if 'behavior' in report:
    bevariour = report['behavior']
    cuckoo_report['generic_behavior'] = bevariour['generic']
    cuckoo_report['apistats'] = np.nan if 'apistats' not in bevariour.keys() else bevariour['apistats']
    cuckoo_report['processes'] = bevariour['processes']
    cuckoo_report['summary'] = {} if 'summary' not in bevariour.keys() else bevariour['summary']

  # Check for suricata analysis and include it if present
  if os.path.exists(os.path.join(reports_path, task_id_name, 'suricata', 'eve.json')):
    with open(os.path.join(reports_path, task_id_name, 'suricata', 'eve.json')) as f:
      eve = json.loads('[' + ','.join(f.readlines()) + ']')
    cuckoo_report['suricata'] = eve[:-1]
    cuckoo_report['suricata_summary'] = eve[-1]

  # Add the parsed report to the list
  cuckoo_reports.append(cuckoo_report)

Skipping .DS_Store as it is not a folder


#### Converting Data to DataFrame

In [4]:
# Convert list of dictionaries to DataFrame and check for missing values.
df = pd.DataFrame(cuckoo_reports)
df.isnull().sum()

task_id                         0
score                           0
signatures_count                0
signature_mark_call_count       0
category                        0
package                         0
strings_count                   0
static                         40
fileops                      1201
generic_behavior               31
apistats                      114
processes                      31
summary                        31
suricata                        4
suricata_summary                4
dtype: int64

In [5]:
# Check data types of DataFrame columns.
df.dtypes

task_id                       object
score                        float64
signatures_count               int64
signature_mark_call_count      int64
category                      object
package                       object
strings_count                  int64
static                        object
fileops                       object
generic_behavior              object
apistats                      object
processes                     object
summary                       object
suricata                      object
suricata_summary              object
dtype: object

In [6]:
# Get value counts of package types in reports.
df.package.value_counts()

package
exe        1851
dll        1024
doc         602
xls          52
7z           35
pdf          19
elf           4
macho         4
ppt           1
default       1
Name: count, dtype: int64

In [7]:
# Get value counts of score ratings in reports.
df.score.value_counts()

score
10.0    3458
8.9       22
2.9       17
0.0       16
9.4       12
9.9       11
9.7        7
3.0        7
9.1        7
9.0        6
9.6        6
9.3        5
5.7        4
8.7        4
0.1        3
3.6        1
4.1        1
6.0        1
3.7        1
0.9        1
3.3        1
6.1        1
3.1        1
Name: count, dtype: int64

<br/><br/>

-----------

<br/><br/>

#### Preprocessing Data for Analysis

In [8]:
# Load malware data and remove rows with missing 'cuckoo_id'.
df_malware = pd.read_csv('malware.csv')
df_malware = df_malware[df_malware.cuckoo_id.notna()]

# Ensure 'cuckoo_id' and 'task_id' are of object type for merging.
df_malware['cuckoo_id'] = df_malware['cuckoo_id'].astype(int).astype(object)
df['task_id'] = df['task_id'].astype(int).astype(object)

# Merge data on 'cuckoo_id' and 'task_id' with a left join.
df_output = df_malware.merge(df, left_on='cuckoo_id', right_on='task_id', how='left')
# Keep rows with non-null 'processes'.
df_output = df_output[df_output.processes.notna()]

df_output

,apt,file,cuckoo_id,task_id,score,signatures_count,signature_mark_call_count,category,package,strings_count,static,fileops,generic_behavior,apistats,processes,summary,suricata,suricata_summary
0,APT 1,4123011354d8259e919fbdf605be1973a79100074959dc...,5121514,5121514,10.0,11,140,archive,exe,195,"{'pdb_path': None, 'pe_imports': [{'imports': ...",NaN,[{'process_path': 'C:\Windows\System32\lsass.e...,"{'172': {'RegCreateKeyExW': 12, 'NtDuplicateOb...",[{'process_path': 'C:\Windows\System32\lsass.e...,{'regkey_written': ['HKEY_CURRENT_USER\Softwar...,[{'timestamp': '2024-08-14T03:28:23.625214+030...,{'timestamp': '2024-08-14T03:37:14.878360+0300...
1,APT 1,c5d3906f7c6f39c16bb9b3d8061026d06f8a6dbb9a363f...,5121628,5121628,10.0,5,0,archive,exe,87,"{'pdb_path': None, 'pe_imports': [{'imports': ...",NaN,[{'process_path': 'C:\Windows\System32\lsass.e...,"{'2908': {'setsockopt': 1, 'WSASocketA': 1, 'S...",[{'process_path': 'C:\Windows\System32\lsass.e...,"{'connects_ip': ['205.159.83.91'], 'file_opene...",[{'timestamp': '2024-08-14T03:46:42.411318+030...,{'timestamp': '2024-08-14T03:55:22.499882+0300...
2,APT 1,6971d8780aafa44664a469ff82074b5fea575b16aad399...,5121432,5121432,10.0,10,44,archive,exe,103,"{'pdb_path': None, 'pe_imports': [{'imports': ...",NaN,[{'process_path': 'C:\Windows\System32\lsass.e...,"{'316': {'RegCreateKeyExW': 5, 'LdrUnloadDll':...",[{'process_path': 'C:\Windows\System32\lsass.e...,{'regkey_written': ['HKEY_CURRENT_USER\Softwar...,[{'timestamp': '2024-08-14T03:14:40.196844+030...,{'timestamp': '2024-08-14T03:23:20.539288+0300...
3,APT 1,f23b384c2f44d7be20389a8f0d9688f650f6b88f8ad370...,5121804,5121804,10.0,16,224,archive,exe,1015,{'pdb_path': 'd:\Projects\WinRAR\SFX\build\sfx...,[{'path': 'C:\Users\Administrator\AppData\Loca...,[{'process_path': 'C:\Users\Administrator\AppD...,"{'1560': {'LdrUnloadDll': 7, 'NtOpenSection': ...",[{'process_path': 'C:\Windows\System32\lsass.e...,{'regkey_deleted': ['HKEY_CURRENT_USER\Softwar...,[{'timestamp': '2024-08-14T04:18:52.074571+030...,{'timestamp': '2024-08-14T04:27:47.251493+0300...
4,APT 1,71434227b085c02a969d16c264574bf49863a0ac8966d0...,5121502,5121502,10.0,10,140,archive,exe,155,"{'pdb_path': None, 'pe_imports': [{'imports': ...",NaN,[{'process_path': 'C:\Windows\System32\lsass.e...,"{'2600': {'RegCreateKeyExW': 12, 'NtDuplicateO...",[{'process_path': 'C:\Windows\System32\lsass.e...,{'regkey_written': ['HKEY_CURRENT_USER\Softwar...,[{'timestamp': '2024-08-14T03:26:09.010023+030...,{'timestamp': '2024-08-14T03:34:56.949981+0300...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3588,Dark Hotel,ab478166ea93e9dac3e37a9ad7457aa58249046003238e...,5127146,5127146,10.0,13,116,archive,exe,2041,"{'pdb_path': None, 'pe_imports': [{'imports': ...",[{'path': 'C:\Users\Administrator\AppData\Loca...,[{'process_path': 'C:\Windows\System32\lsass.e...,"{'2712': {'NtOpenSection': 3, 'RegCloseKey': 1...",[{'process_path': 'C:\Windows\System32\lsass.e...,{'file_created': ['C:\Users\Administrator\AppD...,[{'timestamp': '2024-08-16T12:36:28.915464+030...,{'timestamp': '2024-08-16T12:47:00.150290+0300...
3589,Dark Hotel,9bc2309d5e391dd14c2948c55551105572ec0ae5cfc1f3...,5126365,5126365,10.0,10,132,archive,exe,186,"{'pdb_path': None, 'pe_imports': [{'imports': ...","[{'path': 'C:\Users', 'time': 125, 'op': 'dire...",[{'process_path': 'C:\Windows\System32\lsass.e...,"{'540': {'CreateToolhelp32Snapshot': 3, 'NtDup...",[{'process_path': 'C:\Windows\System32\lsass.e...,{'directory_created': ['C:\Users\Administrator...,[{'timestamp': '2024-08-16T01:14:19.119004+030...,{'timestamp': '2024-08-16T01:22:49.666816+0300...
3590,Dark Hotel,e7d65f2e23e76e2378afe028bd091d98469aa36a3a1bb3...,5127257,5127257,10.0,11,56,archive,exe,172,"{'pdb_path': None, 'pe_imports': [{'imports': ...",NaN,[{'process_path': 'C:\Windows\System32\lsass.e...,"{'2664': {'RegCreateKeyExW': 11, 'NtDuplicateO...",[{'process_path': 'C:\Windows\System32\lsass.e...,{'regkey_written': ['HKEY_CURRENT_USER\So

Optional additional analysis or transformations

In [9]:
# df_temp = df_output[['reboot']]

# df_temp['total_count'] = df_temp['reboot'].apply(lambda x: len(x))
# df_temp['regkey'] = df_temp['reboot'].apply(lambda x: len([i for i in x if 'regkey' in i['category']]))

# df_temp[df_temp['regkey'] != df_temp['total_count']]

</br></br></br>

----

</br></br></br>

#### Extracting API Statistics

In [10]:
def extract_all_api_stats(api):
    # Return an empty dictionary if the input is not a dictionary.
    if not isinstance(api, dict): return {}
    
    compiled_api_stats = {}
    # Compile statistics for each API call across all entries.
    for key in api.keys():
        for api_call, value in api[key].items():
            compiled_api_stats[api_call] = compiled_api_stats.get(api_call, 0) + value

    return compiled_api_stats

# Apply the function to extract API stats and convert results to a DataFrame.
df_ = df_output['apistats'].apply(extract_all_api_stats).apply(pd.Series).fillna(0)

df_

,RegCreateKeyExW,NtDuplicateObject,CoUninitialize,RegCloseKey,NtQueryKey,GetBestInterfaceEx,HttpOpenRequestA,InternetOpenA,WSAStartup,NtClose,...,pdf_unescape,InternetOpenUrlW,WSARecvFrom,WSASendTo,InternetWriteFile,ReadCabinetState,RegisterHotKey,CertCreateCertificateContext,GetInterfaceInfo,CryptProtectData
0,12.0,3.0,5.0,43.0,6.0,2.0,1.0,1.0,7.0,59.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.0,4.0,3.0,30.0,4.0,2.0,0.0,4.0,4.0,41.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,11.0,41.0,41.0,626.0,26.0,42.0,4.0,4.0,16.0,1665.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12.0,4.0,5.0,43.0,6.0,2.0,1.0,1.0,7.0,60.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3588,18.0,10.0,5.0,264.0,8.0,2.0,3.0,3.0,10.0,456.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3589,18.0,9.0,5.0,86.0,8.0,2.0,3.0,3.0,12.0,95.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3590,11.0,9.0,3.0,53.0,4.0,2.0,3.0,3.0,9.0,182.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3591,9.0,8.0,8.0,110.0,4.0,2.0,3.0,3.0,6.0,182.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Performing PCA Analysis

In [11]:
from sklearn.decomposition import PCA
import numpy as np

# Initialize and fit PCA on the DataFrame.
pca = PCA().fit(df_)

# Calculate the importance of each feature based on their contribution to the principal components.
feature_importance = np.abs(pca.components_).sum(axis=0)

# Sort features based on their importance in descending order.
important_features = feature_importance.argsort()[::-1]
sorted_features = df_.columns[important_features]

# Print features ranked by their importance.
print("Features ranked by importance:", sorted_features)


Features ranked by importance: Index(['InternetOpenA', 'DeleteUrlCacheEntryA', 'InternetConnectA',
       'InternetOpenUrlA', 'InternetQueryOptionA', 'HttpSendRequestA',
       'RegCreateKeyExW', 'gethostbyname', 'HttpOpenRequestW', 'PRF',
       ...
       'GetKeyState', 'GetAsyncKeyState', 'NtOpenMutant', 'SetFilePointer',
       'NtDelayExecution', 'recv', 'GetForegroundWindow', '__exception__',
       'GetCursorPos', 'FindResourceExA'],
      dtype='object', length=282)


In [12]:
# Select the top 8 most important features from the sorted list.
important_columns = sorted_features[:8]

#### Final Data Preparation

In [13]:
# Create an empty DataFrame to hold the processed data
df_final = pd.DataFrame()

# Remove '.zip' extension and replace it with '.json'
df_final['file_name'] = df_output['file'].str.replace('.zip', '.json')

# Calculate the number of processes, modules, and calls for each report
df_final['process_call_count'] = df_output['processes'].apply(lambda x: 0 if type(x) != list or len(x) == 0 else sum(0 if 'calls' not in y else len(y['calls']) for y in x))
df_final['system32_module_count'] = df_output['processes'].apply(lambda x: 0 if type(x) != list or len(x) == 0 else sum(len([z for z in data['modules'] if 'system32' in z['filepath'].lower()]) for data in x if 'modules' in data))
df_final['syswow64_module_count'] = df_output['processes'].apply(lambda x: 0 if type(x) != list or len(x) == 0 else sum(len([z for z in data['modules'] if 'syswow64' in z['filepath'].lower()]) for data in x if 'modules' in data))

# Calculate the number of PE resources and imports for each report
df_final['pe_resources_count'] = df_output['static'].apply(lambda x: 0 if type(x) != dict or 'pe_resources' not in x.keys() else len(x['pe_resources']))
df_final['pe_imports_count'] = df_output['static'].apply(lambda x: 0 if type(x) != dict or 'pe_imports' not in x.keys() else sum(len(z) for z in x['pe_imports'] ))

# Count files created under Administrator path from 'fileops'
df_final['files_created_in_administrator'] =  df_output['fileops'].apply(lambda x: 0 if type(x) != list else sum([1 for fileop in x if fileop['op'] == 'file_created' and fileop['path'].startswith('C:\\Users\\Administrator')]))

# Summarize the number of file operations and registry keys accessed
df_final['files_not_ddl_exe_count'] = df_output['generic_behavior'].apply(lambda x: sum(sum(len([ z for z in value if type(z) == str and not z.endswith('.dll') and not z.endswith('.exe')]) for key, value in y.get('summary').items() if 'file' in key and type(value) == list) for y in x))
df_final['explorer_accessed_count'] = df_output['generic_behavior'].apply(lambda x: sum(sum(len([ z for z in value if type(z) == str and 'Windows\\CurrentVersion\\explorer'.lower() in z.lower()]) for key, value in y.get('summary').items() if 'regkey_read' in key and type(value) == list) for y in x))

# Summarize the number of registry keys opened and values accessed
df_final['regkey_opened_count'] = df_output['summary'].apply(lambda x: 0 if type(x) != dict or 'regkey_opened' not in x.keys() else len(x['regkey_opened']))
df_final['summary_values_count'] = df_output['summary'].apply( lambda x: 0 if type(x) != dict else sum(len(val) for val in x.values()))
df_final['keyboard_control_count'] = df_output['summary'].apply(lambda x: 0 if type(x) != dict or 'regkey_opened' not in x.keys() else sum(1 for y in x['regkey_opened'] if 'Keyboard Layouts'.lower() in y.lower()))
df_final['wpad_regkey_count'] = df_output['summary'].apply(lambda x: 0 if type(x) != dict else sum( sum(1 for z in x[y] if type(z) == str and 'wpad' in z.lower()) for y in x.keys()))

# Include the number of strings
df_final['strings_count'] = df_output['strings_count']

# Include the number of signatures and signature mark call count
df_final['signatures_count'] = df_output['signatures_count']
df_final['signature_mark_call_count'] = df_output['signature_mark_call_count']

# Include the top 8 important columns identified by PCA.
for columns in important_columns:
  df_final['api_' + columns] = df_[columns]

# Calculate average packet size from Suricata summary data
df_final['average_pkt_size'] = df_output['suricata_summary'].apply(lambda x: 0 if type(x) != dict else x['stats']['decoder']['avg_pkt_size'])

# Save the final DataFrame to a CSV file
df_final.to_csv('cuckoo_report.csv', index=False) 

df_final

,file_name,process_call_count,system32_module_count,syswow64_module_count,pe_resources_count,pe_imports_count,files_created_in_administrator,files_not_ddl_exe_count,explorer_accessed_count,regkey_opened_count,...,signature_mark_call_count,api_InternetOpenA,api_DeleteUrlCacheEntryA,api_InternetConnectA,api_InternetOpenUrlA,api_InternetQueryOptionA,api_HttpSendRequestA,api_RegCreateKeyExW,api_gethostbyname,average_pkt_size
0,4123011354d8259e919fbdf605be1973a79100074959dc...,0,63,26,1,12,0,3,0,23,...,140,1.0,0.0,1.0,0.0,0.0,3.0,12.0,0.0,133
1,c5d3906f7c6f39c16bb9b3d8061026d06f8a6dbb9a363f...,200,65,20,1,12,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,132
2,6971d8780aafa44664a469ff82074b5fea575b16aad399...,0,62,24,1,10,0,0,0,23,...,44,4.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,270
3,f23b384c2f44d7be20389a8f0d9688f650f6b88f8ad370...,3500,86,67,24,16,16,157,43,295,...,224,4.0,0.0,4.0,0.0,0.0,4.0,11.0,0.0,260
4,71434227b085c02a969d16c264574bf49863a0ac8966d0...,0,63,26,0,12,0,3,0,23,...,140,1.0,0.0,1.0,0.0,0.0,3.0,12.0,0.0,113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3588,ab478166ea93e9dac3e37a9ad7457aa58249046003238e...,1000,68,76,16,6,43,115,0,224,...,116,3.0,0.0,3.0,0.0,0.0,3.0,18.0,3.0,134
3589,9bc2309d5e391dd14c2948c55551105572ec0ae5cfc1f3...,100,64,26,0,18,0,2,0,30,...,132,3.0,0.0,3.0,0.0,0.0,3.0,18.0,3.0,130
3590,e7d65f2e23e76e2378afe028bd091d98469aa36a3a1bb3...,300,64,27,0,20,0,1,0,30,...,56,3.0,0.0,3.0,0.0,0.0,3.0,11.0,3.0,140
3591,962810f908daab4ed0796ff563433eb65a60507d23089a...,300,73,47,19,20,2,8,14,73,...,100,3.0,0.0,3.0,0.0,0.0,3.0,9.0,0.0,137
